# Exercise 3: simulate a Winner-Take-All (WTA) network using DYNAP-SE1 models

By [Jingyue Zhao](https://www.ini.uzh.ch/fw1/modules/ini/ini.php/people/jzhao), [Elisa Donati](https://www.ini.uzh.ch/fw1/modules/ini/ini.php/people/elisa) and 
[Giacomo Indiveri](https://www.ini.uzh.ch/fw1/modules/ini/ini.php/people/giacomo) 

In this exercise we will simulate a Winner-Take-All (WTA) network using DYNAP-SE1 neuron and synapse models. Specifically, we will create a population of neurons connected with local excitation and global inhibition to form a population code. In practice, we often use WTA as one the building blocks of our SNN models. Therefore this exercise of a simulated WTA can be later transformed to a hardware version on DYNAP-SE1.

In this tutorial, we will use 32 excitatory neurons (i.e., the excitatory population) and 8 inhibitory neurons (i.e., the inhibitory population) to create a Winner-Take-All network. As shown in the figure below, local excitation and global inhibition are constructed in a WTA network. 

<center><img src="img/wta.png" alt="WTA circuit." style="width: 700px;"></center>

* Local excitation
    * Feedforward excitatory-inhibition (EI) connections: each excitatory neuron activates the inhibitory neurons with probabilistic connections.
    * Recurrent excitatory-excitatory (EE) connections: each excitatory neuron sends connections to itself (self-excitation) and/or to its neighbors (lateral excitation). Usually, the weights of the recurrent connections are non-decreasing in a sense that neurons that are closer to each other are more correleted.

- Global inhibition
    * Inhibitory-excitatory (IE) connections: each inhibitory neuron is randomly connected to all the excitatory neurons with a certain connection probability.


Thanks to the wonderful dynamics of WTA, it presents some good features, e.g. encoding a variable/state robustly using space coding, signal restoration, cue integration, etc. You can see these functions in Figure 2 of this [paper](https://arxiv.org/abs/1608.08267)).

Let's first import all the required Python libraries.

In [ ]:
from brian2 import *
from DynapSE import DynapSE

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Display plots inside Jupyter cell
%matplotlib inline 
# Set the dots-per-inch (resolution) of the images
mpl.rcParams['figure.dpi'] = 90

from equations.dynapse_eq import *
from parameters.dynapse_param import *

# C++ code generation for faster spiking network simulation
set_device('cpp_standalone')
# Ignore Brian2 base warnings
BrianLogger.suppress_name('base')
# The clock of Brian2 simulation for numerically solve ODEs
defaultclock.dt = 20 * us

## Create neuron populations
We will create one excitatory population with 32 neurons, and one inhibitory neurons with 8 neurons, following the ratio between excitatory neurons and inhibitory neurons in biology.

In [ ]:
# to start a new build to run multiple full simulations
device.reinit()
device.activate()
# need to reset the defaultclock after activate
defaultclock.dt = 20 * us

# create a network
network = Network()
chip = DynapSE(network)

# define the network parameters
# number of exc neurons
num_exc = 32
# number of inh neurons
num_inh = 8

######################## TODO ########################

# create exc population

# create inh population

######################## TODO ########################


## Create spike generators

We then define a group of Poisson spike generators to stimulate the excitatory neurons with one-to-one connections. Here we create 2 bumps of firing rates in the shape of a Gaussian distribution. The first 16 spike generators form a higher peak with 120 Hz in the Gaussian center, while the second half of them form a lower bump with the maximum firing rate at 80 Hz, which mimics the rate distribution of the "Input Stimulus" in page 28 of the lecture slides.

In [ ]:
# Poisson rate of each spike generator
# which forms a population code with a bump

# define the shape of the Gaussian bump
value=0.5
sigma=10
amplitude=200

def gaussian(x, mu, sigma = 1.0):
    '''
    Author: Jingyue
    Calculate the Gaussian value given a position x, the Guassian center mu and sigma.
    '''
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))

def double2pop_code(value, inputSize=16, sigma=1, amplitude=100):
    '''
    Author: Jingyue
    Convert a normalized value in [0,1] to a population code represented by
    a group of neurons. 
    value: normalized value
    inputSize: size of the neuron population
    sigma: Gaussian sigma
    amplitude: firing rate of the Gaussian center
    
    return:
        rates: array of firing rates encoding a normalized variable
    '''
    rates = []
    mu = value*(inputSize - 1)
    
    for i in range(inputSize):
        rate = amplitude * gaussian(float(i), float(mu), float(sigma))
        rates.append(rate)
        
    return rates

rates_bump_1 = double2pop_code(value=0.5, inputSize=int(num_exc/2), sigma=3, amplitude=120)

rates_bump_2 = double2pop_code(value=0.5, inputSize=int(num_exc/2), sigma=3, amplitude=80)

rates = rates_bump_1 + rates_bump_2

# create spike generators for each exc neuron
spikegen = PoissonGroup(num_exc, rates*Hz)


Let's plot the mean firing rates of the Poisson spike generators to visualize the 2 bumps. We will see how the WTA network handle these 2 bumps later.

In [ ]:
plt.plot(np.arange(num_exc), rates, '.-')
plt.title('Mean Firing Rates of the Input Stimulus')
plt.xlabel('Neuron address')    
plt.ylabel('Mean firing rate (Hz)')    

## Create connections
Following the description of the WTA network mentioned above, we will create 4 types of synapses between different neuron groups.
* syn_inp: one-to-one connections from Poisson spike generators to the excitatory neurons
* syn_ee: recurrent excitatory connections inside the excitatory population
* syn_ei: random all-to-all connections from the excitatory population to the inhibitory population with probability = p_ei
* syn_ie: random all-to-all connections from the inhibitory population to the excitatory population with probability = p_ie

For `syn_ee`, we use a Gaussian kernel to model the distance-dreasing weights of the recurrent connections. Let's first take a look at the connection pattern generation. The two arrays `output_i` and `output_j` can be used to create the synapses, e.g. `synapse.connect(i=output_i, j=output_j)`.

In [ ]:
def gaussian_recurrent_excitation(size, baseWeight, sigma=3.0, self_exc=True):
    '''
    Author: Jingyue
    For a pre neuron, use its neuron id as the mean of Gaussian, calculate the value at
    every other neuron id which is the weight of the synapse.
    
    size: number of neurons
    baseWeight: maximum weight at the center
    sigma: Gaussian parameter
    self_exc: if you want to have self-excitation or not
    
    return:
        output_i: pre neuron ids
        output_j: post neuron ids
    '''
    output_i = []
    output_j = []
    floatWeightMr = np.zeros((size,size))
    
    for pre in range(size):
        mu = pre
        for post in range(size):
            if pre == post and self_exc == False:
                continue
            else:
                # for each neuron id, calculate the synapse weight at every other neurons
                weight = int(baseWeight * gaussian(float(post), float(mu), float(sigma)) )
                floatWeightMr[pre][post] = weight

                # add weight connections between pre and post
                for k in range(weight):
                    output_i.append(pre)
                    output_j.append(post)
            
    print(floatWeightMr[16])
        
    return output_i, output_j  

output_i, output_j = gaussian_recurrent_excitation(32, 1, 1, True)    

We can print the weight matrix of neuron 16 in this function. And you can see that with `baseWeight=1, sigma=1, self_exc=True`, neuron 16 only sends 1 connection to itself. 

Let's now create the whole network connections. Here, we first turn off the WTA recurrent and feedback connections to observe the behavior of a pure feedforward network. We only set the weight of syn_inp to be positive, and all the other synaptic weights to be zero.

In [ ]:
# define connection pattern parameters
# probability of EI connections
p_ei = 0.8
# probability of IE connections
p_ie = 0.8

############################## TODO ##############################

# syn_inp, spikegen to exc, one to one, NMDA

# syn_ee, EE: use gaussian_recurrent_excitation(), NMDA

# syn_ei, EI: all to all with probability = 0.8, AMPA

# syn_ie, IE: all to all with probability = 0.8, GABA_B

############################## TODO ##############################

# set weights
syn_inp.weight = 300
syn_ee.weight = 0
syn_ei.weight = 0
syn_ie.weight = 0


## Create monitors and run the simulation
Let's create some spike monitors to observe the input and output firing rates of the spike generators and excitatory neuron group. Then add all the network components into a Brian2 network, and run the simulation. The synaptic parameters are tuned and given here to reproduce the "Feedforward Network" behavior in slide 28.

In [ ]:
# create input monitor
mon_input  = SpikeMonitor(spikegen, name='mon_input')
# create exc monitor
mon_output = SpikeMonitor(exc, name='mon_output')

# tune synaptic parameters
syn_conf = {# nmda
             "I_tau_syn_exc": 2.5 * pA,
             "I_g_syn_exc": 0.3 * pA,
    
             # ampa, default
             "I_tau_syn_exc2": 50 * pA,
             "I_g_syn_exc2": 50 * pA}

chip.set_param(syn_conf, 'Core_1')

network.add([spikegen, exc, inh, syn_inp, syn_ee, syn_ei, syn_ie, mon_input, mon_output])

# Simulation
duration = 10 * second
network.run(duration)

## Observe the behaviors of WTA

Let's save the output firing rates of the excitatory neurons in this feedforward network, and compare them with those of a WTA network later.

In [ ]:
ff_output = mon_output.count/ duration / Hz

Now, we can make a raster plot to visulize the spikes over time which shows the spike timings of each neuron. Also, we can put the mean firing rates of the neurons on the side to directly observe the population code in space.

In [ ]:
# plot the raster plot and mean firing rates of spike generators

fig, (ax1, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]}, figsize=(10,6), sharey=True)
fig.text(0.45, 0.97, "Input Stimulus", ha="center", va="bottom", size="large",color="k")
fig.tight_layout()

ax1.plot(mon_input.t/ms, mon_input.i, '.k')
ax1.set(xlabel='Time (ms)', ylabel='Neuron address')
ax2.plot(mon_input.count/ duration / Hz, np.arange(num_exc), '-k')
ax2.set(xlabel='Mean firing rate (Hz)')

# plot the raster plot and mean firing rates of excitatory neurons in the feedforward network

fig, (ax1, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]}, figsize=(10,6), sharey=True)
fig.text(0.45, 0.97, "Feedforward Network", ha="center", va="bottom", size="large",color="g")
fig.tight_layout()

ax1.plot(mon_output.t/ms, mon_output.i, '.g')
ax1.set(xlabel='Time (ms)', ylabel='Neuron address')
ax2.plot(mon_output.count/ duration / Hz, np.arange(num_exc), '-g')
ax2.set(xlabel='Mean firing rate (Hz)')

Looks familiar right? Check slide 28! 

And now we turn on the WTA connections, i.e., syn_ee, syn_ei, syn_ie. Then if you properly tune the network parameters, you would see the higher peak gets stronger (the winner), and the lower peak is suppressed. You can tune the network to achieve a hard WTA, i.e. only one winner exists, or a soft WTA where the weaker bump are still active. 

Here, can you try to reproduce the "Feedback Network" result on the right-bottom of slide 28?

Hint: you can play with the `gaussian_recurrent_excitation()` to try different recurrent excitation profile, and you can tune the synaptic paremeters to change the dynamics of the network.

In [ ]:
# to start a new build to run multiple full simulations
device.reinit()
device.activate()
# need to reset the defaultclock after activate
defaultclock.dt = 20 * us

# create a network
network = Network()
chip = DynapSE(network)

# define the network parameters
# number of exc neurons
num_exc = 32
# number of inh neurons
num_inh = 8
# probability of EI connections
p_ei = 0.8
# probability of IE connections
p_ie = 0.8

# Create stimulus
rates_bump_1 = double2pop_code(value=0.5, inputSize=int(num_exc/2), sigma=3, amplitude=120)

rates_bump_2 = double2pop_code(value=0.5, inputSize=int(num_exc/2), sigma=3, amplitude=80)

rates = rates_bump_1 + rates_bump_2
    
# create spike generators for each exc neuron
spikegen = PoissonGroup(num_exc, rates*Hz)


######################## TODO ########################

# create exc population

# create inh population

######################## TODO ########################




############################### TODO ###############################

# create synapses, turn on all WTA connections

# syn_inp, spikegen to exc, one to one, NMDA

# syn_ee, EE: play with gaussian_recurrent_excitation(), NMDA

# syn_ei, EI: all to all with probability = 0.8, AMPA

# syn_ie, IE: all to all with probability = 0.8, GABA_B

############################### TODO ###############################

syn_inp.weight = 300
syn_ee.weight = 150
syn_ei.weight = 300
# Note: inhibitory weight should be negative! 
syn_ie.weight = -300


######################## TODO ########################
# tune the synaptic parameters to achieve the desired WTA behavior


######################## TODO ########################



# create input monitor
mon_input  = SpikeMonitor(spikegen, name='mon_input')
# create exc monitor
mon_output = SpikeMonitor(exc, name='mon_output')
# create inh monitor
mon_inh = SpikeMonitor(inh, name='mon_inh')

network.add([spikegen, exc, inh, syn_inp, syn_ee, syn_ei, syn_ie, mon_input, mon_output, mon_inh])

# Simulation
duration = 10 * second
network.run(duration)


We now plot the output firing rates of the excitatory neuron in the feedforward and the WTA network to see the function of the WTA.

In [ ]:
# plot the raster plot and mean firing rates of excitatory neurons in the feedforward network and the WTA network to compare the difference.

fig, (ax1, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]}, figsize=(10,6), sharey=True)
fig.text(0.45, 0.97, "WTA Network", ha="center", va="bottom", size="large",color="b")
fig.tight_layout()

ax1.plot(mon_output.t/ms, mon_output.i, '.b')
ax1.set(xlabel='Time (ms)', ylabel='Neuron address')

ax2.plot(mon_output.count/ duration / Hz, np.arange(num_exc), '-b', label='WTA')
ax2.plot(ff_output, np.arange(num_exc), '-g', label='FF')
ax2.legend(loc="upper right")
ax2.set(xlabel='Mean firing rate (Hz)')

Do you see a difference between the feedforward network and the WTA? 

Let's now plot the activity of the inhibitory neurons as well.

In [ ]:
# plot the raster plot and mean firing rates of inhibitory neurons.

fig, (ax1, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]}, figsize=(10,6), sharey=True)
fig.text(0.45, 0.97, "Inhibitory neurons", ha="center", va="bottom", size="large",color='orange')
fig.tight_layout()

ax1.plot(mon_inh.t/ms, mon_inh.i, '.', color='orange')
ax1.set(xlabel='Time (ms)', ylabel='Neuron address')

ax2.plot(mon_inh.count/ duration / Hz, np.arange(num_inh), '-', color='orange')
ax2.set(xlabel='Mean firing rate (Hz)')

The distribution of firing rates in the inhibitory population are relatively uniform, right? Remember the connections between the excitatory and inhibitory populations? All the excitatory neurons activate the inhibitory neurons, and the inhibitory ones apply global inhibition to the excitatory ones.

Now, can you describe the different behaviors of the feedforward and WTA network and explain why? Please explain the dynamics of the WTA network.

In [ ]:
############################## TODO ###########################
# Explain the dynamics/behaviors of WTA.


############################## TODO ###########################